In [1]:
import pandas as pd
"""
header = 0 은 파일의 첫 번째 줄에 열 이름이 있음을 나타내며 
delimiter = \t 는 필드가 탭으로 구분되는 것을 의미한다.
quoting = 3은 쌍따옴표를 무시하도록 한다.
"""
# QUOTE_MINIMAL (0), QUOTE_ALL (1), 
# QUOTE_NONNUMERIC (2) or QUOTE_NONE (3).

# 레이블인 sentiment 가 있는 학습 데이터
train = pd.read_csv('data/labeledTrainData.tsv', 
                    header=0, delimiter='\t', quoting=3)
# 레이블이 없는 테스트 데이터
test = pd.read_csv('data/testData.tsv', 
                   header=0, delimiter='\t', quoting=3)

In [2]:
# 데이터 전처리는 튜토리얼 파트1~4까지 공통으로 사용되기 때문에 별도의 파이썬 파일로 분리했다.
# 그리고 캐글에 있는 코드를 병렬처리하도록 멀티프로세싱 코드를 추가했다.
# 하지만 여기에서는 멀티프로세싱 코드만 임포트해서 사용하고 전처리는 태그만 제거해주도록 한다.
# 그리고 WordNetLemmatizer로 레마타이징 한다.
# 음소표기법(lemmatization)은 튜토리얼 파트1에 설명 되어 있고 다음 링크에 있다.
# https://github.com/corazzon/KaggleStruggle/blob/master/word2vec-nlp-tutorial/tutorial-part-1.ipynb

from KaggleWord2VecUtility import KaggleWord2VecUtility
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def review_to_words( raw_review ):
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    review_text = wordnet_lemmatizer.lemmatize(review_text)
    return review_text

In [3]:
# 학습데이터를 전처리 한다.
%time train['review_clean'] = train['review'].apply(review_to_words)

Wall time: 7.51 s


In [4]:
# 학습데이터와 동일하게 테스트 데이터에 대해서도 전처리 한다.
%time train['review_clean'] = train['review'].apply(review_to_words)

Wall time: 5.8 s


In [6]:
X_train = train['review_clean']

In [8]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from nltk.corpus import words

vectorizer = CountVectorizer(analyzer = 'word', 
                             lowercase = True,
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = 'english',
                             min_df = 2, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋다. 
                             ngram_range=(1, 3),
                             vocabulary = set(words.words()), # nltk의 words를 사용하거나 문서 자체의 사전을 만들거나 선택한다. 
                             max_features = 90000
                            )

In [9]:
pipeline = Pipeline([
    ('vect', vectorizer),
    ('tfidf', TfidfTransformer(smooth_idf = False)),
])  
%time X_train_tfidf_vector = pipeline.fit_transform(X_train)

D:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1242: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


Wall time: 12.6 s


In [10]:
vocab = vectorizer.get_feature_names()

In [11]:
import numpy as np
dist = np.sum(X_train_tfidf_vector, axis=0)
    
for tag, count in zip(vocab, dist):
    print(count, tag)
    
pd.DataFrame(dist, columns=vocab)

[[0. 0. 0. ... 0. 0. 0.]] A


,A,Aani,Aaron,Aaronic,Aaronical,Aaronite,Aaronitic,Aaru,Ab,Ababdeh,...,zymotechnical,zymotechnics,zymotechny,zymotic,zymotically,zymotize,zymotoxic,zymurgy,zythem,zythum
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
